In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, trim, upper, lower, initcap, current_timestamp, 
    lit, coalesce, when, sum as spark_sum, count, avg, 
    row_number, to_date, date_trunc, percentile_approx,
    collect_list, struct, round as spark_round
)
from pyspark.sql.types import IntegerType, StringType, TimestampType, DateType, FloatType, DecimalType
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F

In [0]:
catalog_name = "atendimento_catalog"
silver_db_name = "silver"
gold_db_name = "gold"

In [0]:
# ============================================================================
# INICIALIZAÇÃO CENTRAL E DETERMINÍSTICA
# ============================================================================
# Esta célula DEVE ser executada ANTES de todas as outras células de transformação.
# Carrega todas as fontes Silver/Bronze necessárias e define variáveis globais.
# Se qualquer tabela estiver faltando, falha imediatamente com mensagem clara.

print("=" * 80)
print("INICIALIZANDO NOTEBOOK GOLD - Carregamento de Fontes Silver")
print("=" * 80)

# ----------------------------------------------------------------------------
# 1. DEFINIÇÃO DE HELPER FUNCTIONS
# ----------------------------------------------------------------------------

def assert_instantiated(var_name: str, obj):
    """
    Valida que um DataFrame ou variável foi instanciado corretamente.
    Falha imediatamente se o objeto for None ou inválido.
    """
    if obj is None:
        raise RuntimeError(
            f"ERRO CRÍTICO: Variável '{var_name}' não foi instanciada corretamente. "
            f"Execute a célula de inicialização primeiro."
        )
    return obj

def safe_table_exists(table_name: str) -> bool:
    """Verifica se uma tabela existe no catálogo."""
    try:
        spark.table(table_name)
        return True
    except Exception:
        return False

def _safe_count(df):
    """Retorna contagem segura de registros."""
    try:
        return df.count()
    except Exception:
        return 0

# ----------------------------------------------------------------------------
# 2. CARREGAMENTO DE TODAS AS FONTES SILVER (FAIL-FAST)
# ----------------------------------------------------------------------------

# Definir tabelas necessárias com os nomes REAIS das tabelas Silver
required_silver_tables = {
    'satisfacao': f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao",
    'chamados': f"{catalog_name}.{silver_db_name}.ft_chamados",
    'clientes': f"{catalog_name}.{silver_db_name}.dm_clientes",
    'canais': f"{catalog_name}.{silver_db_name}.dm_canais",
    'motivos': f"{catalog_name}.{silver_db_name}.dm_motivos",
}

# Tabelas opcionais (não críticas para todas as análises)
optional_silver_tables = {
    'atendentes': f"{catalog_name}.{silver_db_name}.ft_atendentes",
    'custos': f"{catalog_name}.{silver_db_name}.ft_custos",
}

# Verificar existência de todas as tabelas ANTES de carregar
print("\n[1/4] Verificando existência de tabelas Silver...")
missing_tables = []
for name, table_path in required_silver_tables.items():
    exists = safe_table_exists(table_path)
    status = "✓" if exists else "✗"
    print(f"  {status} {name}: {table_path}")
    if not exists:
        missing_tables.append(table_path)

if missing_tables:
    error_msg = (
        f"\n{'=' * 80}\n"
        f"ERRO CRÍTICO: Tabelas Silver não encontradas no catálogo:\n"
        f"{chr(10).join('  - ' + t for t in missing_tables)}\n"
        f"{'=' * 80}\n"
        f"AÇÃO NECESSÁRIA:\n"
        f"1. Execute o notebook de transformação Bronze → Silver primeiro\n"
        f"2. Verifique que o catálogo '{catalog_name}' e schema '{silver_db_name}' existem\n"
        f"3. Confirme que as tabelas foram criadas corretamente\n"
        f"   Tabelas esperadas: ft_pesquisa_satisfacao, ft_chamados, dm_clientes, dm_canais, dm_motivos\n"
        f"{'=' * 80}"
    )
    raise RuntimeError(error_msg)

print("\n[2/4] Carregando DataFrames Silver...")

# Carregar satisfacao (fonte para satisfaction_trends)
df_satisfacao_src = spark.table(required_silver_tables['satisfacao'])
print(f"  ✓ df_satisfacao_src: {_safe_count(df_satisfacao_src):,} registros")

# Carregar chamados
df_chamados_src = spark.table(required_silver_tables['chamados'])
print(f"  ✓ df_chamados_src: {_safe_count(df_chamados_src):,} registros")

# Carregar clientes
df_clientes_src = spark.table(required_silver_tables['clientes'])
print(f"  ✓ df_clientes_src: {_safe_count(df_clientes_src):,} registros")

# Carregar canais
df_canais_src = spark.table(required_silver_tables['canais'])
print(f"  ✓ df_canais_src: {_safe_count(df_canais_src):,} registros")

# Carregar motivos
df_motivos_src = spark.table(required_silver_tables['motivos'])
print(f"  ✓ df_motivos_src: {_safe_count(df_motivos_src):,} registros")

# Carregar tabelas opcionais (se disponíveis)
if safe_table_exists(optional_silver_tables['atendentes']):
    df_atendentes_src = spark.table(optional_silver_tables['atendentes'])
    print(f"  ✓ df_atendentes_src: {_safe_count(df_atendentes_src):,} registros")
else:
    df_atendentes_src = None
    print(f"  ⚠ df_atendentes_src: não disponível (opcional)")

if safe_table_exists(optional_silver_tables['custos']):
    df_custos_src = spark.table(optional_silver_tables['custos'])
    print(f"  ✓ df_custos_src: {_safe_count(df_custos_src):,} registros")
else:
    df_custos_src = None
    print(f"  ⚠ df_custos_src: não disponível (opcional)")

# Criar aliases para compatibilidade com código downstream
df_satisfacao_trends_src = df_satisfacao_src
df_chamados_trends_src = df_chamados_src

# ----------------------------------------------------------------------------
# 3. CONSTRUÇÃO DE df_base_satisfaction (DataFrame Central)
# ----------------------------------------------------------------------------

print("\n[3/4] Construindo df_base_satisfaction (joins + schema canonical)...")

try:
    # Validar que todos os DataFrames necessários existem
    assert_instantiated('df_satisfacao_trends_src', df_satisfacao_trends_src)
    assert_instantiated('df_chamados_trends_src', df_chamados_trends_src)
    assert_instantiated('df_clientes_src', df_clientes_src)
    assert_instantiated('df_canais_src', df_canais_src)
    assert_instantiated('df_motivos_src', df_motivos_src)
    
    # Construir DataFrame base com joins
    df_base_satisfaction = (
        df_satisfacao_trends_src.alias("s")
        .join(
            df_chamados_trends_src.alias("c"),
            col("s.id_chamado") == col("c.id_chamado"),
            "left"
        )
        .join(
            df_clientes_src.alias("cl"),
            col("c.id_cliente") == col("cl.id_cliente"),
            "left"
        )
        .join(
            df_canais_src.alias("ca"),
            col("c.id_canal") == col("ca.id_canal"),
            "left"
        )
        .join(
            df_motivos_src.alias("m"),
            col("c.id_motivo") == col("m.id_motivo"),
            "left"
        )
        .select(
            col("s.id_chamado"),
            col("c.id_cliente"),
            col("s.nota_atendimento").alias("nota_satisfacao"),
            col("c.hora_abertura_chamado"),
            col("c.tempo_espera_minutos"),
            col("cl.regiao").alias("regiao"),
            col("ca.nome_canal").alias("canal"),
            col("m.nome_motivo").alias("motivo"),
        )
        .withColumn("data", to_date(col("hora_abertura_chamado")))
        .withColumn("semana", F.weekofyear(col("hora_abertura_chamado")))
    )
    
    # Validar colunas obrigatórias
    required_columns = ['id_chamado', 'id_cliente', 'nota_satisfacao', 'data', 'semana', 
                       'canal', 'regiao', 'motivo', 'hora_abertura_chamado', 'tempo_espera_minutos']
    missing_cols = [c for c in required_columns if c not in df_base_satisfaction.columns]
    if missing_cols:
        raise RuntimeError(f"df_base_satisfaction está faltando colunas obrigatórias: {missing_cols}")
    
    print(f"  ✓ df_base_satisfaction: {_safe_count(df_base_satisfaction):,} registros")
    
except Exception as e:
    raise RuntimeError(f"Falha ao construir df_base_satisfaction: {e}")

# ----------------------------------------------------------------------------
# 4. CONSTRUÇÃO DE df_with_fcr (First Contact Resolution)
# ----------------------------------------------------------------------------

print("\n[4/4] Construindo df_with_fcr (métricas de primeira interação)...")

try:
    # Começar com df_base_satisfaction
    df_with_fcr = df_base_satisfaction
    
    # Tentar trazer colunas de resolução e sequência de chamados
    chamados_cols = df_chamados_src.columns
    
    additional_cols = ['id_chamado']
    if 'resolvido' in chamados_cols:
        additional_cols.append('resolvido')
    if 'chamado_seq' in chamados_cols:
        additional_cols.append('chamado_seq')
    
    # Join apenas se houver colunas extras
    if len(additional_cols) > 1:
        df_ch_extra = df_chamados_src.select(*additional_cols)
        df_with_fcr = df_with_fcr.join(df_ch_extra, on='id_chamado', how='left')
    
    # Computar flag_primeira_interacao
    if 'chamado_seq' in df_with_fcr.columns:
        df_with_fcr = df_with_fcr.withColumn(
            'flag_primeira_interacao',
            when(col('chamado_seq') == 1, 1).otherwise(0).cast('int')
        )
    else:
        # Fallback: assumir que todas são primeira interação (padrão seguro)
        df_with_fcr = df_with_fcr.withColumn('flag_primeira_interacao', lit(1).cast('int'))
    
    # Normalizar coluna 'resolvido'
    if 'resolvido' in df_with_fcr.columns:
        df_with_fcr = df_with_fcr.withColumn(
            'resolvido',
            when(col('resolvido').isNull(), lit('Nao')).otherwise(col('resolvido'))
        )
    else:
        df_with_fcr = df_with_fcr.withColumn('resolvido', lit('Nao'))
    
    # Computar flag_resolvido_primeira_interacao
    df_with_fcr = df_with_fcr.withColumn(
        'flag_resolvido_primeira_interacao',
        when(
            (col('flag_primeira_interacao') == 1) & (col('resolvido') == 'Sim'),
            1
        ).otherwise(0).cast('int')
    )
    
    print(f"  ✓ df_with_fcr: {_safe_count(df_with_fcr):,} registros")
    
except Exception as e:
    raise RuntimeError(f"Falha ao construir df_with_fcr: {e}")

# ----------------------------------------------------------------------------
# 5. DEFINIÇÃO DE VARIÁVEIS GLOBAIS (Nomes de Tabelas Gold)
# ----------------------------------------------------------------------------

print("\nDefinindo variáveis globais para tabelas Gold...")

# Nomes de tabelas Gold (destino das transformações)
tgt_table_funnel = f"{catalog_name}.{gold_db_name}.funnel_metrics"
tgt_table_anomalous = f"{catalog_name}.{gold_db_name}.anomalous_escalations"
tgt_table_top_volume = f"{catalog_name}.{gold_db_name}.top_reasons_volume"
tgt_table_top_cost = f"{catalog_name}.{gold_db_name}.top_reasons_cost"
tgt_table_top_satisfaction = f"{catalog_name}.{gold_db_name}.top_reasons_low_satisfaction"
tgt_table_satisfaction_trends = f"{catalog_name}.{gold_db_name}.satisfaction_trends"
tgt_table_satisfaction_by_canal = f"{catalog_name}.{gold_db_name}.satisfaction_by_canal"
tgt_table_satisfaction_by_regiao = f"{catalog_name}.{gold_db_name}.satisfaction_by_regiao"
tgt_table_operational_metrics = f"{catalog_name}.{gold_db_name}.operational_metrics"

# Contadores para relatórios (pré-computados)
total_before_satisfacao_trends = _safe_count(df_satisfacao_src)
total_before_chamados = _safe_count(df_chamados_src)
total_before_clientes = _safe_count(df_clientes_src)
total_before_canais = _safe_count(df_canais_src)
total_before_motivos = _safe_count(df_motivos_src)
total_before_atendentes = _safe_count(df_atendentes_src) if df_atendentes_src is not None else 0
total_before_custos = _safe_count(df_custos_src) if df_custos_src is not None else 0

# ----------------------------------------------------------------------------
# 6. RESUMO DA INICIALIZAÇÃO
# ----------------------------------------------------------------------------

print("\n" + "=" * 80)
print("✓ INICIALIZAÇÃO COMPLETA - Resumo:")
print("=" * 80)
print(f"  DataFrames Silver carregados:")
print(f"  - df_satisfacao_src: {total_before_satisfacao_trends:,}")
print(f"  - df_chamados_src: {total_before_chamados:,}")
print(f"  - df_clientes_src: {total_before_clientes:,}")
print(f"  - df_canais_src: {total_before_canais:,}")
print(f"  - df_motivos_src: {total_before_motivos:,}")
if df_atendentes_src is not None:
    print(f"  - df_atendentes_src: {total_before_atendentes:,}")
if df_custos_src is not None:
    print(f"  - df_custos_src: {total_before_custos:,}")
print(f"\n  DataFrames derivados:")
print(f"  - df_base_satisfaction: {_safe_count(df_base_satisfaction):,}")
print(f"  - df_with_fcr: {_safe_count(df_with_fcr):,}")
print(f"\n  Variáveis de tabelas Gold definidas: 8")
print("=" * 80)
print("O notebook está pronto para executar células de transformação.")
print("=" * 80 + "\n")


## Funções Utilitárias

Funções auxiliares para validação e transformação de dados.

In [0]:
def safe_col(df, name: str):
    if name in df.columns:
        return col(name)
    else:
        return lit(None)


## Processamento: funnel_metrics

Agregações de métricas de funil com análise de conversão e custos.

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias da camada Silver.

### Enriquecimento com Dimensões

Join das tabelas de fato com dimensões para análise completa.

In [0]:
# Validar que DataFrames necessários foram inicializados
assert_instantiated('df_chamados_src', df_chamados_src)
assert_instantiated('df_canais_src', df_canais_src)
assert_instantiated('df_clientes_src', df_clientes_src)
assert_instantiated('df_motivos_src', df_motivos_src)

# Verificar se df_atendentes_src e df_custos_src existem (opcionais para algumas análises)
try:
    df_atendentes_src
except NameError:
    print("AVISO: df_atendentes_src não disponível. Algumas métricas de atendentes serão limitadas.")
    df_atendentes_src = None

try:
    df_custos_src
except NameError:
    print("AVISO: df_custos_src não disponível. Análises de custo serão limitadas.")
    df_custos_src = None

try:
    df_satisfacao_src
except NameError:
    print("AVISO: df_satisfacao_src não disponível. Usando df_base_satisfaction para satisfação.")
    df_satisfacao_src = None

# Construir df_base apenas se tiver dados necessários
if df_atendentes_src is not None and df_custos_src is not None and df_satisfacao_src is not None:
    df_base = df_chamados_src \
        .join(df_atendentes_src, df_chamados_src.id_atendente == df_atendentes_src.id_atendente, "left") \
        .join(df_canais_src, df_chamados_src.id_canal == df_canais_src.id_canal, "left") \
        .join(df_clientes_src, df_chamados_src.id_cliente == df_clientes_src.id_cliente, "left") \
        .join(df_motivos_src, df_chamados_src.id_motivo == df_motivos_src.id_motivo, "left") \
        .join(df_custos_src, df_chamados_src.id_chamado == df_custos_src.id_chamado, "left") \
        .join(df_satisfacao_src, df_chamados_src.id_chamado == df_satisfacao_src.id_chamado, "left") \
        .select(
            df_chamados_src["id_chamado"],
            df_chamados_src["id_cliente"],
            df_chamados_src["id_canal"],
            df_chamados_src["id_atendente"],
            df_chamados_src["id_motivo"],
            df_chamados_src["hora_abertura_chamado"],
            df_chamados_src["hora_inicio_atendimento"],
            df_chamados_src["hora_finalizacao_atendimento"],
            safe_col(df_chamados_src, "resolvido").alias("resolvido"),
            safe_col(df_chamados_src, "canal").alias("canal_chamado"),
            safe_col(df_chamados_src, "tempo_espera_minutos").alias("tempo_espera_minutos"),
            safe_col(df_chamados_src, "tempo_atendimento_minutos").alias("tempo_atendimento_minutos"),
            df_canais_src["nome_canal"].alias("canal"),
            df_atendentes_src["nivel_atendimento"],
            safe_col(df_clientes_src, "regiao").alias("regiao"),
            df_motivos_src["nome_motivo"].alias("motivo"),
            df_custos_src["custo"],
            safe_col(df_satisfacao_src, "nota_atendimento").alias("nota_satisfacao")
        )
    print("df_base construído com sucesso para análise de funnel completo")
else:
    print("AVISO: Usando df_base_satisfaction como fallback (algumas colunas podem estar faltando)")
    df_base = df_base_satisfaction


### Classificação de Etapas do Funil

Categorização de cada chamado em etapas do funil.

In [0]:
# Adicionar colunas temporais (data, semana, ano) ANTES da classificação de etapas
df_funnel = df_base.withColumn("data", to_date(col("hora_abertura_chamado")))
df_funnel = df_funnel.withColumn("semana", F.weekofyear(col("hora_abertura_chamado")))
df_funnel = df_funnel.withColumn("ano", F.year(col("hora_abertura_chamado")))

print("Colunas temporais adicionadas: data, semana, ano")

# Classificação de etapas do funil
df_funnel = df_funnel.withColumn(
    "n_prevencao",
    lit(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_autosservico",
    when(
        (col("canal").isin(["Ura", "Chatbot", "App"])) & 
        (col("id_atendente").isNull()) &
        (col("hora_finalizacao_atendimento").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel1",
    when(
        (col("nivel_atendimento") == 1) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel2",
    when(
        (col("nivel_atendimento") == 2) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_resolvido",
    when(
        col("resolvido") == "Sim",
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "flag_missing_timestamps",
    when(
        col("hora_abertura_chamado").isNull() |
        col("hora_inicio_atendimento").isNull() |
        col("hora_finalizacao_atendimento").isNull(),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "flag_escalacao_incorreta",
    when(
        (col("nivel_atendimento").isNotNull()) &
        (col("n_autosservico") == 0) &
        (col("canal").isin(["Ura", "Chatbot", "App"])),
        1
    ).otherwise(0).cast(IntegerType())
)

print("Classificação de etapas aplicada")


### Cálculo de Reabertura

Identificação de chamados repetidos por cliente.

In [0]:
w_cliente = Window.partitionBy("id_cliente", "data").orderBy(col("hora_abertura_chamado"))

df_funnel = df_funnel.withColumn(
    "chamado_seq",
    row_number().over(w_cliente)
)

df_funnel = df_funnel.withColumn(
    "flag_reabertura",
    when(col("chamado_seq") > 1, 1).otherwise(0).cast(IntegerType())
)

print("Cálculo de reabertura aplicado")

### Agregação por Data, Semana, Canal e Região

Agregação de métricas com múltiplas dimensões.

In [0]:
df_aggregated = df_funnel.groupBy("data", "semana", "ano", "canal", "regiao").agg(
    spark_sum("n_prevencao").alias("n_prevencao"),
    spark_sum("n_autosservico").alias("n_autosservico"),
    spark_sum("n_nivel1").alias("n_nivel1"),
    spark_sum("n_nivel2").alias("n_nivel2"),
    spark_sum("n_resolvido").alias("n_resolvido"),
    count("id_chamado").alias("volume_total_chamados"),
    spark_sum("flag_missing_timestamps").alias("count_missing_timestamps"),
    spark_sum("flag_reabertura").alias("count_reabertura"),
    spark_sum("flag_escalacao_incorreta").alias("count_escalacao_incorreta"),
    spark_sum("custo").alias("custo_total")
)

print(f"Registros agregados: {df_aggregated.count():,}")

### Cálculo de Taxas de Conversão

Cálculo de taxas de conversão entre etapas.

In [0]:
df_with_conversion = df_aggregated.withColumn(
    "taxa_conversao_autosservico_nivel1",
    when(
        col("n_autosservico") > 0,
        F.round((col("n_nivel1") / col("n_autosservico")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_conversion = df_with_conversion.withColumn(
    "taxa_conversao_nivel1_nivel2",
    when(
        col("n_nivel1") > 0,
        F.round((col("n_nivel2") / col("n_nivel1")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_conversion = df_with_conversion.withColumn(
    "taxa_conversao_entre_etapas",
    F.round((col("taxa_conversao_autosservico_nivel1") + col("taxa_conversao_nivel1_nivel2")) / 2, 2).cast(FloatType())
)

df_with_conversion = df_with_conversion.withColumn(
    "taxa_conversao_autosservico_resolvido",
    when(
        col("n_autosservico") > 0,
        F.round((col("n_resolvido") / col("n_autosservico")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

print("Taxas de conversão calculadas")

### Cálculo de Percentuais

Cálculo de percentuais de missing e reabertura.

In [0]:
df_with_pct = df_with_conversion.withColumn(
    "pct_missing_timestamps",
    when(
        col("volume_total_chamados") > 0,
        F.round((col("count_missing_timestamps") / col("volume_total_chamados")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_pct = df_with_pct.withColumn(
    "taxa_reabertura",
    when(
        col("volume_total_chamados") > 0,
        F.round((col("count_reabertura") / col("volume_total_chamados")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_pct = df_with_pct.withColumn(
    "pct_escalacao_incorreta",
    when(
        col("volume_total_chamados") > 0,
        F.round((col("count_escalacao_incorreta") / col("volume_total_chamados")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

print("Percentuais calculados")

### Cálculo de Custo por Etapa

Cálculo de custo por etapa utilizando valores reais.

In [0]:
df_with_cost = df_with_pct.withColumn(
    "custo_por_etapa",
    when(
        col("volume_total_chamados") > 0,
        F.round(col("custo_total") / col("volume_total_chamados"), 2)
    ).otherwise(lit(None)).cast(DecimalType(10, 2))
)

print("Custos calculados")

### Preparação Final

Adição de metadados de processamento e seleção de colunas finais.

In [0]:
df_funnel_final = df_with_cost.withColumn("processed_timestamp", current_timestamp())

final_cols_funnel = [
    "data",
    "semana",
    "ano",
    "canal",
    "regiao",
    "n_prevencao",
    "n_autosservico",
    "n_nivel1",
    "n_nivel2",
    "taxa_conversao_entre_etapas",
    "taxa_conversao_autosservico_resolvido",
    "volume_total_chamados",
    "custo_por_etapa",
    "pct_missing_timestamps",
    "taxa_reabertura",
    "pct_escalacao_incorreta",
    "processed_timestamp"
]

df_funnel_final = df_funnel_final.select(*[c for c in final_cols_funnel if c in df_funnel_final.columns])

df_funnel_typed = df_funnel_final \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("n_prevencao", col("n_prevencao").cast(IntegerType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_nivel2", col("n_nivel2").cast(IntegerType())) \
    .withColumn("taxa_conversao_entre_etapas", col("taxa_conversao_entre_etapas").cast(FloatType())) \
    .withColumn("taxa_conversao_autosservico_resolvido", col("taxa_conversao_autosservico_resolvido").cast(FloatType())) \
    .withColumn("volume_total_chamados", col("volume_total_chamados").cast(IntegerType())) \
    .withColumn("custo_por_etapa", col("custo_por_etapa").cast(DecimalType(10, 2))) \
    .withColumn("pct_missing_timestamps", col("pct_missing_timestamps").cast(FloatType())) \
    .withColumn("taxa_reabertura", col("taxa_reabertura").cast(FloatType())) \
    .withColumn("pct_escalacao_incorreta", col("pct_escalacao_incorreta").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros finais: {df_funnel_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela transformada em formato Delta.

In [0]:
df_funnel_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_funnel)

gold_table_funnel = spark.table(tgt_table_funnel)
final_count_funnel = gold_table_funnel.count()
duplicates_funnel = gold_table_funnel.groupBy("data", "canal", "regiao").count().filter(col("count") > 1).count()

print(f"Salvo em: {tgt_table_funnel}")
print(f"Contagem final: {final_count_funnel:,}")
print(f"Duplicatas na Gold: {duplicates_funnel}")

display(gold_table_funnel.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FUNNEL_METRICS")
print("="*80)
print(f"Registros Silver (origem chamados): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_funnel:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: anomalous_escalations

Identificação de casos com escalações anômalas.

### Identificação de Escalações Anômalas

Detecção de casos que foram escalados incorretamente.

In [0]:
tgt_table_anomalous = f"{catalog_name}.{gold_db_name}.anomalous_escalations"

df_anomalous = df_funnel.filter(
    col("flag_escalacao_incorreta") == 1
).select(
    col("id_chamado"),
    col("id_cliente"),
    col("data"),
    col("canal"),
    col("nivel_atendimento"),
    col("n_autosservico"),
    col("n_nivel1"),
    col("n_nivel2"),
    col("motivo"),
    lit("Escalado sem autosservico").alias("tipo_anomalia")
).withColumn("processed_timestamp", current_timestamp())

df_anomalous_typed = df_anomalous \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("nivel_atendimento", col("nivel_atendimento").cast(IntegerType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_nivel2", col("n_nivel2").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("tipo_anomalia", col("tipo_anomalia").cast(StringType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de escalações anômalas: {df_anomalous_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de escalações anômalas.

In [0]:
df_anomalous_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_anomalous)

gold_table_anomalous = spark.table(tgt_table_anomalous)
final_count_anomalous = gold_table_anomalous.count()

print(f"Salvo em: {tgt_table_anomalous}")
print(f"Contagem final: {final_count_anomalous:,}")

display(gold_table_anomalous.limit(10))

### Relatório de Transformação

Resumo estatístico de escalações anômalas.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - ANOMALOUS_ESCALATIONS")
print("="*80)
print(f"Registros Silver (origem): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_anomalous:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: top_reasons_volume

Top motivos por volume de chamados.

### Agregação por Motivo

Contagem de chamados por motivo.

In [0]:
tgt_table_top_volume = f"{catalog_name}.{gold_db_name}.top_reasons_volume"

df_top_volume = df_funnel.groupBy("motivo").agg(
    count("id_chamado").alias("volume_chamados")
).orderBy(col("volume_chamados").desc())

df_top_volume = df_top_volume.withColumn("processed_timestamp", current_timestamp())

df_top_volume_typed = df_top_volume \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_volume_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por volume.

In [0]:
df_top_volume_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_volume)

gold_table_top_volume = spark.table(tgt_table_top_volume)
final_count_top_volume = gold_table_top_volume.count()

print(f"Salvo em: {tgt_table_top_volume}")
print(f"Contagem final: {final_count_top_volume:,}")

display(gold_table_top_volume.limit(10))

## Processamento: top_reasons_cost

Top motivos por custo total.

### Agregação por Motivo e Custo

Soma de custos por motivo.

In [0]:
tgt_table_top_cost = f"{catalog_name}.{gold_db_name}.top_reasons_cost"

df_top_cost = df_funnel.groupBy("motivo").agg(
    spark_sum("custo").alias("custo_total"),
    count("id_chamado").alias("volume_chamados")
).orderBy(col("custo_total").desc())

df_top_cost = df_top_cost.withColumn(
    "custo_medio",
    when(
        col("volume_chamados") > 0,
        F.round(col("custo_total") / col("volume_chamados"), 2)
    ).otherwise(lit(None)).cast(DecimalType(10, 2))
)

df_top_cost = df_top_cost.withColumn("processed_timestamp", current_timestamp())

df_top_cost_typed = df_top_cost \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("custo_total", col("custo_total").cast(DecimalType(18, 8))) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("custo_medio", col("custo_medio").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_cost_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por custo.

In [0]:
df_top_cost_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_cost)

gold_table_top_cost = spark.table(tgt_table_top_cost)
final_count_top_cost = gold_table_top_cost.count()

print(f"Salvo em: {tgt_table_top_cost}")
print(f"Contagem final: {final_count_top_cost:,}")

display(gold_table_top_cost.limit(10))

## Processamento: top_reasons_low_satisfaction

Top motivos por baixa satisfação.

### Agregação por Motivo e Satisfação

Média de satisfação por motivo.

In [0]:
tgt_table_top_satisfaction = f"{catalog_name}.{gold_db_name}.top_reasons_low_satisfaction"

df_with_satisfaction = df_funnel.filter(col("nota_satisfacao").isNotNull())

df_top_satisfaction = df_with_satisfaction.groupBy("motivo").agg(
    avg("nota_satisfacao").alias("nota_media"),
    count("id_chamado").alias("volume_chamados"),
    spark_sum(
        when(col("nota_satisfacao") <= 3, 1).otherwise(0)
    ).alias("count_baixa_satisfacao")
).orderBy(col("nota_media").asc())

df_top_satisfaction = df_top_satisfaction.withColumn(
    "pct_baixa_satisfacao",
    when(
        col("volume_chamados") > 0,
        F.round((col("count_baixa_satisfacao") / col("volume_chamados")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_top_satisfaction = df_top_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_top_satisfaction_typed = df_top_satisfaction \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("nota_media", F.round(col("nota_media"), 2).cast(FloatType())) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("pct_baixa_satisfacao", col("pct_baixa_satisfacao").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_satisfaction_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por baixa satisfação.

In [0]:
df_top_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_satisfaction)

gold_table_top_satisfaction = spark.table(tgt_table_top_satisfaction)
final_count_top_satisfaction = gold_table_top_satisfaction.count()

print(f"Salvo em: {tgt_table_top_satisfaction}")
print(f"Contagem final: {final_count_top_satisfaction:,}")

display(gold_table_top_satisfaction.limit(10))

## Processamento: client_profiles

Perfil agregado por cliente.

### Agregação por Cliente

Cálculo de métricas por cliente.

In [0]:
tgt_table_client_profiles = f"{catalog_name}.{gold_db_name}.client_profiles"

w_motivo = Window.partitionBy("id_cliente").orderBy(col("count_motivo").desc())

df_motivo_por_cliente = df_funnel.groupBy("id_cliente", "motivo").agg(
    count("id_chamado").alias("count_motivo")
)

df_top_motivo = df_motivo_por_cliente \
    .withColumn("rn", row_number().over(w_motivo)) \
    .filter(col("rn") == 1) \
    .select("id_cliente", col("motivo").alias("top_motivo"))

df_client_profiles = df_funnel.groupBy("id_cliente").agg(
    count("id_chamado").alias("n_chamados"),
    avg("custo").alias("avg_custo"),
    avg("nota_satisfacao").alias("avg_satisfaction")
)

df_client_profiles = df_client_profiles.join(
    df_top_motivo,
    df_client_profiles.id_cliente == df_top_motivo.id_cliente,
    "left"
).select(
    df_client_profiles["id_cliente"],
    df_client_profiles["n_chamados"],
    df_client_profiles["avg_custo"],
    df_client_profiles["avg_satisfaction"],
    df_top_motivo["top_motivo"]
)

df_client_profiles = df_client_profiles.withColumn("processed_timestamp", current_timestamp())

df_client_profiles_typed = df_client_profiles \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("n_chamados", col("n_chamados").cast(IntegerType())) \
    .withColumn("avg_custo", F.round(col("avg_custo"), 2).cast(DecimalType(10, 2))) \
    .withColumn("avg_satisfaction", F.round(col("avg_satisfaction"), 2).cast(FloatType())) \
    .withColumn("top_motivo", col("top_motivo").cast(StringType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de clientes únicos: {df_client_profiles_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de perfis de clientes.

In [0]:
df_client_profiles_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_client_profiles)

gold_table_client_profiles = spark.table(tgt_table_client_profiles)
final_count_client_profiles = gold_table_client_profiles.count()

print(f"Salvo em: {tgt_table_client_profiles}")
print(f"Contagem final: {final_count_client_profiles:,}")

display(gold_table_client_profiles.limit(10))

### Relatório de Transformação

Resumo estatístico de perfis de clientes.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - CLIENT_PROFILES")
print("="*80)
print(f"Registros Silver (origem clientes): {total_before_clientes:,}")
print(f"Registros Gold (destino): {final_count_client_profiles:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

In [0]:
catalog_name = "atendimento_catalog"
silver_db_name = "silver"
gold_db_name = "gold"

In [0]:
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_db_name}")
spark.sql(f"USE SCHEMA {gold_db_name}")

## Funções Utilitárias

Funções auxiliares para validação e transformação de dados.
Reutilização de funções do padrão Silver.

In [0]:
def safe_table_exists(spark, full_name: str) -> bool:
    try:
        spark.table(full_name)
        return True
    except AnalysisException:
        return False
    except Exception:
        return False

## Processamento: funnel_metrics

Transformação da camada Silver para Gold com agregações de métricas de funil.
Cálculo de conversões entre etapas e custos operacionais por canal.

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias da camada Silver.
Validação de existência e contagem inicial de registros.

In [0]:
src_table_chamados = f"{catalog_name}.{silver_db_name}.ft_chamados"
src_table_atendentes = f"{catalog_name}.{silver_db_name}.ft_atendentes"
src_table_canais = f"{catalog_name}.{silver_db_name}.dm_canais"
src_table_custos = f"{catalog_name}.{silver_db_name}.ft_custos"
tgt_table_funnel = f"{catalog_name}.{gold_db_name}.funnel_metrics"

if not safe_table_exists(spark, src_table_chamados):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_chamados}")
if not safe_table_exists(spark, src_table_atendentes):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_atendentes}")
if not safe_table_exists(spark, src_table_canais):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_canais}")
if not safe_table_exists(spark, src_table_custos):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_custos}")

df_chamados_src = spark.table(src_table_chamados)
df_atendentes_src = spark.table(src_table_atendentes)
df_canais_src = spark.table(src_table_canais)
df_custos_src = spark.table(src_table_custos)

total_before_chamados = df_chamados_src.count()
total_before_atendentes = df_atendentes_src.count()
total_before_canais = df_canais_src.count()
total_before_custos = df_custos_src.count()

print(f"Leitura: {src_table_chamados}")
print(f"Registros ft_chamados: {total_before_chamados:,}")
print(f"Registros ft_atendentes: {total_before_atendentes:,}")
print(f"Registros dm_canais: {total_before_canais:,}")
print(f"Registros ft_custos: {total_before_custos:,}")

### Enriquecimento com Dimensões

Join das tabelas de fato com dimensões para obter nomes de canais e níveis de atendimento.
Preparação de dataset base para agregações.

In [0]:
df_base = df_chamados_src \
    .join(df_atendentes_src, df_chamados_src.id_atendente == df_atendentes_src.id_atendente, "left") \
    .join(df_canais_src, df_chamados_src.id_canal == df_canais_src.id_canal, "left") \
    .join(df_custos_src, df_chamados_src.id_chamado == df_custos_src.id_chamado, "left") \
    .select(
        df_chamados_src["id_chamado"],
        df_chamados_src["id_cliente"],
        df_chamados_src["id_canal"],
        df_chamados_src["id_atendente"],
        df_chamados_src["hora_abertura_chamado"],
        df_chamados_src["hora_inicio_atendimento"],
        df_chamados_src["hora_finalizacao_atendimento"],
        df_canais_src["nome_canal"].alias("canal"),
        df_atendentes_src["nivel_atendimento"],
        coalesce(df_custos_src["custo"], lit(0.0)).alias("custo")
    )

df_base = df_base.withColumn(
    "data",
    to_date(col("hora_abertura_chamado"))
)

print(f"Dataset base enriquecido: {df_base.count():,} registros")

### Classificação de Etapas do Funil

Categorização de cada chamado em etapas do funil baseado em canal e nível de atendimento.
Identificação de autosserviço, nível 1 e nível 2.

In [0]:
df_funnel = df_base.withColumn(
    "n_prevencao",
    lit(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_autosservico",
    when(
        (col("canal").isin(["Ura", "Chatbot", "App"])) & 
        (col("id_atendente").isNull()) &
        (col("hora_finalizacao_atendimento").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel1",
    when(
        (col("nivel_atendimento") == 1) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel2",
    when(
        (col("nivel_atendimento") == 2) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

print("Classificação de etapas aplicada")

### Agregação por Data e Canal

Agregação de métricas por data e canal com contadores de cada etapa do funil.
Cálculo de totais para análise de conversão.

In [0]:
df_aggregated = df_funnel.groupBy("data", "canal").agg(
    spark_sum("n_prevencao").alias("n_prevencao"),
    spark_sum("n_autosservico").alias("n_autosservico"),
    spark_sum("n_nivel1").alias("n_nivel1"),
    spark_sum("n_nivel2").alias("n_nivel2"),
    count("id_chamado").alias("total_chamados"),
    spark_sum("custo").alias("custo_total")
)

print(f"Registros agregados: {df_aggregated.count():,}")

### Cálculo de Taxa de Conversão

Cálculo de taxas de conversão entre etapas sequenciais do funil.
Conversões expressas em percentual.

In [0]:
df_with_conversion = df_aggregated.withColumn(
    "taxa_conversao_autosservico_nivel1",
    when(
        col("n_autosservico") > 0,
        F.round((col("n_nivel1") / col("n_autosservico")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_conversion = df_with_conversion.withColumn(
    "taxa_conversao_nivel1_nivel2",
    when(
        col("n_nivel1") > 0,
        F.round((col("n_nivel2") / col("n_nivel1")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_conversion = df_with_conversion.withColumn(
    "taxa_conversao_entre_etapas",
    F.round((col("taxa_conversao_autosservico_nivel1") + col("taxa_conversao_nivel1_nivel2")) / 2, 2).cast(FloatType())
)

print("Taxas de conversão calculadas")

### Cálculo de Custo por Etapa

Cálculo de custo por etapa utilizando valores reais da tabela custos.

In [0]:
df_with_cost = df_with_conversion.withColumn(
    "custo_por_etapa",
    when(
        col("total_chamados") > 0,
        F.round(col("custo_total") / col("total_chamados"), 2)
    ).otherwise(lit(None)).cast(DecimalType(10, 2))
)

print("Custos calculados")

### Preparação Final

Adição de metadados de processamento e seleção de colunas finais.
Aplicação de schema com constraints.

In [0]:
df_funnel_final = df_with_cost.withColumn("processed_timestamp", current_timestamp())

final_cols_funnel = [
    "data",
    "canal",
    "n_prevencao",
    "n_autosservico",
    "n_nivel1",
    "n_nivel2",
    "taxa_conversao_entre_etapas",
    "custo_por_etapa",
    "processed_timestamp"
]

df_funnel_final = df_funnel_final.select(*[c for c in final_cols_funnel if c in df_funnel_final.columns])

df_funnel_typed = df_funnel_final \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("n_prevencao", col("n_prevencao").cast(IntegerType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_nivel2", col("n_nivel2").cast(IntegerType())) \
    .withColumn("taxa_conversao_entre_etapas", col("taxa_conversao_entre_etapas").cast(FloatType())) \
    .withColumn("custo_por_etapa", col("custo_por_etapa").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros finais: {df_funnel_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela transformada em formato Delta na camada Gold.
Modo overwrite com overwriteSchema para garantir schema atualizado.

In [0]:
df_funnel_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_funnel)

gold_table_funnel = spark.table(tgt_table_funnel)
final_count_funnel = gold_table_funnel.count()
duplicates_funnel = gold_table_funnel.groupBy("data", "canal").count().filter(col("count") > 1).count()

print(f"Salvo em: {tgt_table_funnel}")
print(f"Contagem final: {final_count_funnel:,}")
print(f"Duplicatas na Gold: {duplicates_funnel}")

display(gold_table_funnel.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.
Métricas de qualidade e distribuição por canal.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FUNNEL_METRICS")
print("="*80)
print(f"Registros Silver (origem chamados): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_funnel:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: satisfaction_trends

Análise de tendências de satisfação com correlação de tempo de espera.

## Inicialização de DataFrames Base

Criação antecipada de DataFrames reutilizados em múltiplas seções Gold.

In [0]:
# Carregamento antecipado das tabelas Silver necessárias para criar df_base_satisfaction
src_table_satisfacao_trends = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
src_table_chamados_trends = f"{catalog_name}.{silver_db_name}.ft_chamados"
src_table_clientes_trends = f"{catalog_name}.{silver_db_name}.dm_clientes"
src_table_canais_trends = f"{catalog_name}.{silver_db_name}.dm_canais"
src_table_motivos_trends = f"{catalog_name}.{silver_db_name}.dm_motivos"

if not safe_table_exists(spark, src_table_satisfacao_trends):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_satisfacao_trends}")
if not safe_table_exists(spark, src_table_chamados_trends):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_chamados_trends}")
if not safe_table_exists(spark, src_table_clientes_trends):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_clientes_trends}")
if not safe_table_exists(spark, src_table_canais_trends):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_canais_trends}")
if not safe_table_exists(spark, src_table_motivos_trends):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_motivos_trends}")

df_satisfacao_trends_src = spark.table(src_table_satisfacao_trends)
df_chamados_trends_src = spark.table(src_table_chamados_trends)
df_clientes_trends_src = spark.table(src_table_clientes_trends)
df_canais_trends_src = spark.table(src_table_canais_trends)
df_motivos_trends_src = spark.table(src_table_motivos_trends)

# Criação de df_base_satisfaction (reutilizado em satisfaction_trends, top_motivos_*, satisfaction_by_*)
df_base_satisfaction = df_satisfacao_trends_src \
    .join(df_chamados_trends_src, df_satisfacao_trends_src.id_chamado == df_chamados_trends_src.id_chamado, "left") \
    .join(df_clientes_trends_src, df_chamados_trends_src.id_cliente == df_clientes_trends_src.id_cliente, "left") \
    .join(df_canais_trends_src, df_chamados_trends_src.id_canal == df_canais_trends_src.id_canal, "left") \
    .join(df_motivos_trends_src, df_chamados_trends_src.id_motivo == df_motivos_trends_src.id_motivo, "left") \
    .select(
        df_satisfacao_trends_src["id_chamado"],
        df_chamados_trends_src["id_cliente"],
        safe_col(df_satisfacao_trends_src, "nota_atendimento").alias("nota_satisfacao"),
        df_chamados_trends_src["hora_abertura_chamado"],
        safe_col(df_chamados_trends_src, "tempo_espera_minutos").alias("tempo_espera_minutos"),
        safe_col(df_clientes_trends_src, "regiao").alias("regiao"),
        df_canais_trends_src["nome_canal"].alias("canal"),
        df_motivos_trends_src["nome_motivo"].alias("motivo")
    )

df_base_satisfaction = df_base_satisfaction \
    .withColumn("data", to_date(col("hora_abertura_chamado"))) \
    .withColumn("semana", F.weekofyear(col("hora_abertura_chamado")))

# Validações de schema
required_columns = ["id_chamado", "id_cliente", "nota_satisfacao", "hora_abertura_chamado", 
                    "tempo_espera_minutos", "regiao", "canal", "motivo", "data", "semana"]
missing_cols = [c for c in required_columns if c not in df_base_satisfaction.columns]
if missing_cols:
    raise RuntimeError(f"df_base_satisfaction está faltando colunas: {missing_cols}")

print(f"✓ df_base_satisfaction criado: {df_base_satisfaction.count():,} registros")
print(f"✓ Schema validado: {', '.join(df_base_satisfaction.columns)}")

## Processamento: top_motivos_baixa_nota

Identificação dos principais motivos de baixa satisfação.

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias da camada Silver (reutiliza df_base_satisfaction criado em satisfaction_trends).

### Agregação por Motivo

Filtro e ranking de motivos com baixa satisfação.

In [0]:
df_top_baixa_nota = df_base_satisfaction \
    .filter(col("nota_satisfacao") <= 3) \
    .groupBy("motivo").agg(
        count("id_chamado").alias("volume_baixa_nota"),
        avg("nota_satisfacao").alias("nota_media_baixa")
    ) \
    .orderBy(col("volume_baixa_nota").desc()) \
    .limit(10)

print(f"Top 10 motivos de baixa satisfação: {df_top_baixa_nota.count():,}")

### Preparação Final

Adição de metadados de processamento e tipagem de colunas.

In [0]:
df_top_baixa_nota = df_top_baixa_nota.withColumn("processed_timestamp", current_timestamp())

df_top_baixa_nota_typed = df_top_baixa_nota \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("volume_baixa_nota", col("volume_baixa_nota").cast(IntegerType())) \
    .withColumn("nota_media_baixa", F.round(col("nota_media_baixa"), 2).cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos processados: {df_top_baixa_nota_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela transformada em formato Delta.

In [0]:
df_satisfaction_agg = df_base_satisfaction.groupBy(
    "data", "semana", "regiao", "canal", "motivo"
).agg(
    count("id_chamado").alias("n_total_respostas"),
    avg("nota_satisfacao").alias("nota_media"),
    percentile_approx("nota_satisfacao", 0.5).alias("nota_mediana"),
    spark_sum(
        when(col("nota_satisfacao") <= 3, 1).otherwise(0)
    ).alias("count_baixa_satisfacao"),
    avg("tempo_espera_minutos").alias("tempo_espera_medio"),
    collect_list(
        struct(
            col("tempo_espera_minutos").alias("wait"),
            col("nota_satisfacao").alias("score")
        )
    ).alias("wait_score_pairs")
)

df_satisfaction_agg = df_satisfaction_agg.withColumn(
    "pct_baixa_satisfacao",
    when(
        col("n_total_respostas") > 0,
        spark_round((col("count_baixa_satisfacao") / col("n_total_respostas")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

display(df_satisfaction_agg)

### Cálculo de Correlação Tempo de Espera vs Nota

Cálculo de correlação Pearson entre tempo de espera e satisfação.

In [0]:
df_correlation = df_base_satisfaction \
    .filter(col("tempo_espera_minutos").isNotNull() & col("nota_satisfacao").isNotNull()) \
    .groupBy("data", "semana", "regiao", "canal", "motivo").agg(
        F.corr("tempo_espera_minutos", "nota_satisfacao").alias("correlation_wait_vs_score"),
        count("id_chamado").alias("n_points")
    )

df_correlation = df_correlation.withColumn(
    "correlation_wait_vs_score",
    when(
        (col("n_points") >= 2) & col("correlation_wait_vs_score").isNotNull(),
        F.round(col("correlation_wait_vs_score"), 4).cast(FloatType())
    ).otherwise(lit(None).cast(FloatType()))
)

df_correlation = df_correlation.drop("n_points")

print(f"Correlações calculadas: {df_correlation.count():,}")

### Merge de Correlação com Agregação

Combinação de métricas de agregação com correlação.

In [0]:
df_satisfaction_final = df_satisfaction_agg.join(
    df_correlation,
    ["data", "semana", "regiao", "canal", "motivo"],
    "left"
).select(
    col("data"),
    col("semana"),
    col("regiao"),
    col("canal"),
    col("motivo"),
    col("n_total_respostas"),
    col("nota_media"),
    col("nota_mediana"),
    col("pct_baixa_satisfacao"),
    col("tempo_espera_medio"),
    col("correlation_wait_vs_score")
).drop("wait_score_pairs")

print(f"Total de registros após merge: {df_satisfaction_final.count():,}")

### Preparação Final

Adição de metadados de processamento e tipagem de colunas.

In [0]:
df_satisfaction_final = df_satisfaction_final.withColumn("processed_timestamp", current_timestamp())

df_satisfaction_typed = df_satisfaction_final \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("n_total_respostas", col("n_total_respostas").cast(IntegerType())) \
    .withColumn("nota_media", F.round(col("nota_media"), 2).cast(FloatType())) \
    .withColumn("nota_mediana", col("nota_mediana").cast(FloatType())) \
    .withColumn("pct_baixa_satisfacao", col("pct_baixa_satisfacao").cast(FloatType())) \
    .withColumn("tempo_espera_medio", F.round(col("tempo_espera_medio"), 2).cast(FloatType())) \
    .withColumn("correlation_wait_vs_score", col("correlation_wait_vs_score").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros finais: {df_satisfaction_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela transformada em formato Delta.

In [0]:
tgt_table_satisfaction_trends = f"{catalog_name}.{gold_db_name}.satisfaction_trends"

In [0]:
df_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_trends)

gold_table_satisfaction_trends = spark.table(tgt_table_satisfaction_trends)
final_count_satisfaction_trends = gold_table_satisfaction_trends.count()

print(f"Salvo em: {tgt_table_satisfaction_trends}")
print(f"Contagem final: {final_count_satisfaction_trends:,}")

display(gold_table_satisfaction_trends.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - SATISFACTION_TRENDS")
print("="*80)
print(f"Registros Silver (origem satisfacao): {total_before_satisfacao_trends:,}")
print(f"Registros Gold (destino): {final_count_satisfaction_trends:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Processamento: top_motivos_baixa_nota

Top motivos por baixa satisfação com análise detalhada.

### Agregação por Motivo e Baixa Satisfação

Ranking de motivos com pior satisfação.

In [0]:
tgt_table_top_baixa_nota = f"{catalog_name}.{gold_db_name}.top_motivos_baixa_nota"

df_top_baixa_nota = df_base_satisfaction \
    .filter(col("nota_satisfacao").isNotNull()) \
    .groupBy("motivo").agg(
        avg("nota_satisfacao").alias("nota_media"),
        count("id_chamado").alias("volume_respostas"),
        spark_sum(
            when(col("nota_satisfacao") <= 3, 1).otherwise(0)
        ).alias("count_baixa_satisfacao")
    ).orderBy(col("nota_media").asc())

df_top_baixa_nota = df_top_baixa_nota.withColumn(
    "pct_baixa_satisfacao",
    when(
        col("volume_respostas") > 0,
        F.round((col("count_baixa_satisfacao") / col("volume_respostas")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_top_baixa_nota = df_top_baixa_nota.withColumn("processed_timestamp", current_timestamp())

df_top_baixa_nota_typed = df_top_baixa_nota \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("nota_media", F.round(col("nota_media"), 2).cast(FloatType())) \
    .withColumn("volume_respostas", col("volume_respostas").cast(IntegerType())) \
    .withColumn("pct_baixa_satisfacao", col("pct_baixa_satisfacao").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_baixa_nota_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por baixa nota.

In [0]:
df_top_baixa_nota_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_baixa_nota)

gold_table_top_baixa_nota = spark.table(tgt_table_top_baixa_nota)
final_count_top_baixa_nota = gold_table_top_baixa_nota.count()

print(f"Salvo em: {tgt_table_top_baixa_nota}")
print(f"Contagem final: {final_count_top_baixa_nota:,}")

display(gold_table_top_baixa_nota.limit(10))

## Processamento: top_motivos_volume_satisfaction

Top motivos por volume e satisfação combinados.

### Agregação por Motivo com Volume e Satisfação

Ranking de motivos combinando volume de chamados e satisfação.

In [0]:
tgt_table_top_volume_satisfaction = f"{catalog_name}.{gold_db_name}.top_motivos_volume_satisfaction"

df_top_volume_satisfaction = df_base_satisfaction \
    .filter(col("nota_satisfacao").isNotNull()) \
    .groupBy("motivo").agg(
        count("id_chamado").alias("volume_chamados"),
        avg("nota_satisfacao").alias("nota_media"),
        spark_sum(
            when(col("nota_satisfacao") <= 3, 1).otherwise(0)
        ).alias("count_baixa_satisfacao")
    )

df_top_volume_satisfaction = df_top_volume_satisfaction.withColumn(
    "pct_baixa_satisfacao",
    when(
        col("volume_chamados") > 0,
        F.round((col("count_baixa_satisfacao") / col("volume_chamados")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_top_volume_satisfaction = df_top_volume_satisfaction.withColumn(
    "score_combinado",
    F.round(
        col("volume_chamados") * (1 - (col("nota_media") / 5.0)),
        2
    ).cast(FloatType())
).orderBy(col("score_combinado").desc())

df_top_volume_satisfaction = df_top_volume_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_top_volume_satisfaction_typed = df_top_volume_satisfaction \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("nota_media", F.round(col("nota_media"), 2).cast(FloatType())) \
    .withColumn("pct_baixa_satisfacao", col("pct_baixa_satisfacao").cast(FloatType())) \
    .withColumn("score_combinado", col("score_combinado").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_volume_satisfaction_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de top motivos por volume e satisfação.

In [0]:
df_top_volume_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_volume_satisfaction)

gold_table_top_volume_satisfaction = spark.table(tgt_table_top_volume_satisfaction)
final_count_top_volume_satisfaction = gold_table_top_volume_satisfaction.count()

print(f"Salvo em: {tgt_table_top_volume_satisfaction}")
print(f"Contagem final: {final_count_top_volume_satisfaction:,}")

display(gold_table_top_volume_satisfaction.limit(10))

## Processamento: satisfaction_by_canal

Agregação de satisfação por canal.

### Agregação por Canal

Métricas de satisfação agregadas por canal.

In [0]:
tgt_table_satisfaction_by_canal = f"{catalog_name}.{gold_db_name}.satisfaction_by_canal"

df_satisfaction_by_canal = df_base_satisfaction \
    .filter(col("nota_satisfacao").isNotNull()) \
    .groupBy("canal").agg(
        count("id_chamado").alias("volume_respostas"),
        avg("nota_satisfacao").alias("nota_media"),
        percentile_approx("nota_satisfacao", 0.5).alias("nota_mediana"),
        spark_sum(
            when(col("nota_satisfacao") <= 3, 1).otherwise(0)
        ).alias("count_baixa_satisfacao")
    )

df_satisfaction_by_canal = df_satisfaction_by_canal.withColumn(
    "pct_baixa_satisfacao",
    when(
        col("volume_respostas") > 0,
        F.round((col("count_baixa_satisfacao") / col("volume_respostas")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_satisfaction_by_canal = df_satisfaction_by_canal.withColumn("processed_timestamp", current_timestamp())

df_satisfaction_by_canal_typed = df_satisfaction_by_canal \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("volume_respostas", col("volume_respostas").cast(IntegerType())) \
    .withColumn("nota_media", F.round(col("nota_media"), 2).cast(FloatType())) \
    .withColumn("nota_mediana", col("nota_mediana").cast(FloatType())) \
    .withColumn("pct_baixa_satisfacao", col("pct_baixa_satisfacao").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de canais únicos: {df_satisfaction_by_canal_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de satisfação por canal.

In [0]:
df_satisfaction_by_canal_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_by_canal)

gold_table_satisfaction_by_canal = spark.table(tgt_table_satisfaction_by_canal)
final_count_satisfaction_by_canal = gold_table_satisfaction_by_canal.count()

print(f"Salvo em: {tgt_table_satisfaction_by_canal}")
print(f"Contagem final: {final_count_satisfaction_by_canal:,}")

display(gold_table_satisfaction_by_canal.limit(10))

## Processamento: satisfaction_by_regiao

Agregação de satisfação por região.

### Agregação por Região

Métricas de satisfação agregadas por região.

In [0]:
tgt_table_satisfaction_by_regiao = f"{catalog_name}.{gold_db_name}.satisfaction_by_regiao"
tgt_table_operational_metrics = f"{catalog_name}.{gold_db_name}.operational_metrics"

df_satisfaction_by_regiao = df_base_satisfaction \
    .filter(col("nota_satisfacao").isNotNull()) \
    .groupBy("regiao").agg(
        count("id_chamado").alias("volume_respostas"),
        avg("nota_satisfacao").alias("nota_media"),
        percentile_approx("nota_satisfacao", 0.5).alias("nota_mediana"),
        spark_sum(
            when(col("nota_satisfacao") <= 3, 1).otherwise(0)
        ).alias("count_baixa_satisfacao")
    )

df_satisfaction_by_regiao = df_satisfaction_by_regiao.withColumn(
    "pct_baixa_satisfacao",
    when(
        col("volume_respostas") > 0,
        F.round((col("count_baixa_satisfacao") / col("volume_respostas")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_satisfaction_by_regiao = df_satisfaction_by_regiao.withColumn("processed_timestamp", current_timestamp())

df_satisfaction_by_regiao_typed = df_satisfaction_by_regiao \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("volume_respostas", col("volume_respostas").cast(IntegerType())) \
    .withColumn("nota_media", F.round(col("nota_media"), 2).cast(FloatType())) \
    .withColumn("nota_mediana", col("nota_mediana").cast(FloatType())) \
    .withColumn("pct_baixa_satisfacao", col("pct_baixa_satisfacao").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de regiões únicas: {df_satisfaction_by_regiao_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela de satisfação por região.

In [0]:
df_satisfaction_by_regiao_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_by_regiao)

gold_table_satisfaction_by_regiao = spark.table(tgt_table_satisfaction_by_regiao)
final_count_satisfaction_by_regiao = gold_table_satisfaction_by_regiao.count()

print(f"Salvo em: {tgt_table_satisfaction_by_regiao}")
print(f"Contagem final: {final_count_satisfaction_by_regiao:,}")

display(gold_table_satisfaction_by_regiao.limit(10))

## Processamento: satisfaction_trends

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias da camada Silver.

### Agregação por Data, Semana, Região, Canal e Motivo

Cálculo de métricas de satisfação com múltiplas dimensões.

In [0]:
try:
    total_before_satisfacao_trends = df_satisfacao_trends_src.count()
except NameError:
    try:
        total_before_satisfacao_trends = df_satisfacao_src.count()
    except NameError:
        total_before_satisfacao_trends = df_base_satisfaction.count()

print(f"Registros Silver (origem satisfacao): {total_before_satisfacao_trends:,}")


In [0]:
df_satisfaction_agg = df_base_satisfaction.groupBy("data", "semana", "regiao", "canal", "motivo").agg(
    count("id_chamado").alias("n_total_respostas"),
    avg("nota_satisfacao").alias("nota_media"),
    percentile_approx("nota_satisfacao", 0.5).alias("nota_mediana"),
    spark_sum(
        when(col("nota_satisfacao") <= 3, 1).otherwise(0)
    ).alias("count_baixa_satisfacao"),
    avg("tempo_espera_minutos").alias("tempo_espera_medio"),
    F.collect_list(
        F.struct(
            col("tempo_espera_minutos").alias("wait"),
            col("nota_satisfacao").alias("score")
        )
    ).alias("wait_score_pairs")
)

df_satisfaction_agg = df_satisfaction_agg.withColumn(
    "pct_baixa_satisfacao",
    when(
        col("n_total_respostas") > 0,
        F.round((col("count_baixa_satisfacao") / col("n_total_respostas")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

print(f"Registros agregados: {df_satisfaction_agg.count():,}")

### Cálculo de Correlação Tempo de Espera vs Nota

Cálculo de correlação Pearson entre tempo de espera e satisfação.

In [0]:
df_operational_metrics = df_with_fcr.groupBy("data", "semana", "canal", "regiao").agg(
    count("id_chamado").alias("volume_chamados"),
    avg("tempo_espera_minutos").alias("tempo_espera_medio"),
    spark_sum("flag_primeira_interacao").alias("count_primeira_interacao"),
    spark_sum("flag_resolvido_primeira_interacao").alias("count_resolvido_primeira_interacao"),
    spark_sum(
        when(col("resolvido") == "Sim", 1).otherwise(0)
    ).alias("count_resolvido")
)

df_operational_metrics = df_operational_metrics.withColumn(
    "taxa_resolucao_primeira_interacao",
    when(
        col("count_primeira_interacao") > 0,
        F.round((col("count_resolvido_primeira_interacao") / col("count_primeira_interacao")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_operational_metrics = df_operational_metrics.withColumn(
    "taxa_resolucao_geral",
    when(
        col("volume_chamados") > 0,
        F.round((col("count_resolvido") / col("volume_chamados")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

print(f"Registros agregados: {df_operational_metrics.count():,}")

In [0]:
df_correlation = df_base_satisfaction \
    .filter(col("tempo_espera_minutos").isNotNull() & col("nota_satisfacao").isNotNull()) \
    .groupBy("data", "semana", "regiao", "canal", "motivo").agg(
        F.corr("tempo_espera_minutos", "nota_satisfacao").alias("correlation_wait_vs_score"),
        count("id_chamado").alias("n_points")
    )

df_correlation = df_correlation.withColumn(
    "correlation_wait_vs_score",
    when(
        (col("n_points") >= 2) & col("correlation_wait_vs_score").isNotNull(),
        F.round(col("correlation_wait_vs_score"), 4).cast(FloatType())
    ).otherwise(lit(None).cast(FloatType()))
)

df_correlation = df_correlation.drop("n_points")

print(f"Correlações calculadas: {df_correlation.count():,}")

### Preparação Final

Adição de metadados de processamento e tipagem de colunas.

### Merge de Correlação com Agregação

Combinação de métricas de agregação com correlação.

In [0]:
df_satisfaction_final = df_satisfaction_agg.join(
    df_correlation,
    ["data", "semana", "regiao", "canal", "motivo"],
    "left"
).select(
    col("data"),
    col("semana"),
    col("regiao"),
    col("canal"),
    col("motivo"),
    col("n_total_respostas"),
    col("nota_media"),
    col("nota_mediana"),
    col("pct_baixa_satisfacao"),
    col("tempo_espera_medio"),
    col("correlation_wait_vs_score")
).drop("wait_score_pairs")

print(f"Total de registros após merge: {df_satisfaction_final.count():,}")

In [0]:
df_satisfaction_final = df_satisfaction_final.withColumn("processed_timestamp", current_timestamp())

df_satisfaction_typed = df_satisfaction_final \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("n_total_respostas", col("n_total_respostas").cast(IntegerType())) \
    .withColumn("nota_media", F.round(col("nota_media"), 2).cast(FloatType())) \
    .withColumn("nota_mediana", col("nota_mediana").cast(FloatType())) \
    .withColumn("pct_baixa_satisfacao", col("pct_baixa_satisfacao").cast(FloatType())) \
    .withColumn("tempo_espera_medio", F.round(col("tempo_espera_medio"), 2).cast(FloatType())) \
    .withColumn("correlation_wait_vs_score", col("correlation_wait_vs_score").cast(FloatType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros finais: {df_satisfaction_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela transformada em formato Delta.

In [0]:
df_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_trends)

gold_table_satisfaction_trends = spark.table(tgt_table_satisfaction_trends)
final_count_satisfaction_trends = gold_table_satisfaction_trends.count()

print(f"Salvo em: {tgt_table_satisfaction_trends}")
print(f"Contagem final: {final_count_satisfaction_trends:,}")

display(gold_table_satisfaction_trends.limit(10))

## Processamento: operational_metrics

Métricas operacionais de atendimento.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - OPERATIONAL_METRICS")
print("="*80)
print(f"Registros Silver (origem chamados): {total_before_chamados:,}")
print(f"Registros Gold (destino): {df_operational_metrics.count():,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias para métricas operacionais.

In [0]:
# Tabelas Silver já carregadas no Cell 3 (initialization):
# - df_chamados_src, df_canais_src, df_clientes_src
# tgt_table_operational_metrics definido no Cell 3
# total_before_chamados pré-computado no Cell 3
pass

## Funções Utilitárias

Funções auxiliares para validação e transformação de dados.
Reutilização de funções do padrão Silver.

## Processamento: funnel_metrics

Transformação da camada Silver para Gold com agregações de métricas de funil.
Cálculo de conversões entre etapas e custos operacionais por canal.

### Leitura das Tabelas Silver

Carregamento das tabelas necessárias da camada Silver.
Validação de existência e contagem inicial de registros.

In [0]:
src_table_chamados = f"{catalog_name}.{silver_db_name}.ft_chamados"
src_table_atendentes = f"{catalog_name}.{silver_db_name}.ft_atendentes"
src_table_canais = f"{catalog_name}.{silver_db_name}.dm_canais"
src_table_custos = f"{catalog_name}.{silver_db_name}.ft_custos"
tgt_table_funnel = f"{catalog_name}.{gold_db_name}.funnel_metrics"

if not safe_table_exists(spark, src_table_chamados):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_chamados}")
if not safe_table_exists(spark, src_table_atendentes):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_atendentes}")
if not safe_table_exists(spark, src_table_canais):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_canais}")
if not safe_table_exists(spark, src_table_custos):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_custos}")

df_chamados_src = spark.table(src_table_chamados)
df_atendentes_src = spark.table(src_table_atendentes)
df_canais_src = spark.table(src_table_canais)
df_custos_src = spark.table(src_table_custos)

total_before_chamados = df_chamados_src.count()
total_before_atendentes = df_atendentes_src.count()
total_before_canais = df_canais_src.count()
total_before_custos = df_custos_src.count()

print(f"Leitura: {src_table_chamados}")
print(f"Registros ft_chamados: {total_before_chamados:,}")
print(f"Registros ft_atendentes: {total_before_atendentes:,}")
print(f"Registros dm_canais: {total_before_canais:,}")
print(f"Registros ft_custos: {total_before_custos:,}")

### Enriquecimento com Dimensões

Join das tabelas de fato com dimensões para obter nomes de canais e níveis de atendimento.
Preparação de dataset base para agregações.

In [0]:
df_base = df_chamados_src \
    .join(df_atendentes_src, df_chamados_src.id_atendente == df_atendentes_src.id_atendente, "left") \
    .join(df_canais_src, df_chamados_src.id_canal == df_canais_src.id_canal, "left") \
    .join(df_custos_src, df_chamados_src.id_chamado == df_custos_src.id_chamado, "left") \
    .select(
        df_chamados_src["id_chamado"],
        df_chamados_src["id_cliente"],
        df_chamados_src["id_canal"],
        df_chamados_src["id_atendente"],
        df_chamados_src["hora_abertura_chamado"],
        df_chamados_src["hora_inicio_atendimento"],
        df_chamados_src["hora_finalizacao_atendimento"],
        df_canais_src["nome_canal"].alias("canal"),
        df_atendentes_src["nivel_atendimento"],
        coalesce(df_custos_src["custo"], lit(0.0)).alias("custo")
    )

df_base = df_base.withColumn(
    "data",
    to_date(col("hora_abertura_chamado"))
)

print(f"Dataset base enriquecido: {df_base.count():,} registros")

### Classificação de Etapas do Funil

Categorização de cada chamado em etapas do funil baseado em canal e nível de atendimento.
Identificação de autosserviço, nível 1 e nível 2.

In [0]:
df_funnel = df_base.withColumn(
    "n_prevencao",
    lit(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_autosservico",
    when(
        (col("canal").isin(["Ura", "Chatbot", "App"])) & 
        (col("id_atendente").isNull()) &
        (col("hora_finalizacao_atendimento").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel1",
    when(
        (col("nivel_atendimento") == 1) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

df_funnel = df_funnel.withColumn(
    "n_nivel2",
    when(
        (col("nivel_atendimento") == 2) & (col("id_atendente").isNotNull()),
        1
    ).otherwise(0).cast(IntegerType())
)

print("Classificação de etapas aplicada")

### Agregação por Data e Canal

Agregação de métricas por data e canal com contadores de cada etapa do funil.
Cálculo de totais para análise de conversão.

In [0]:
df_aggregated = df_funnel.groupBy("data", "canal").agg(
    spark_sum("n_prevencao").alias("n_prevencao"),
    spark_sum("n_autosservico").alias("n_autosservico"),
    spark_sum("n_nivel1").alias("n_nivel1"),
    spark_sum("n_nivel2").alias("n_nivel2"),
    count("id_chamado").alias("total_chamados"),
    spark_sum("custo").alias("custo_total")
)

print(f"Registros agregados: {df_aggregated.count():,}")

### Cálculo de Taxa de Conversão

Cálculo de taxas de conversão entre etapas sequenciais do funil.
Conversões expressas em percentual.

In [0]:
df_with_conversion = df_aggregated.withColumn(
    "taxa_conversao_autosservico_nivel1",
    when(
        col("n_autosservico") > 0,
        F.round((col("n_nivel1") / col("n_autosservico")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_conversion = df_with_conversion.withColumn(
    "taxa_conversao_nivel1_nivel2",
    when(
        col("n_nivel1") > 0,
        F.round((col("n_nivel2") / col("n_nivel1")) * 100, 2)
    ).otherwise(lit(0.0)).cast(FloatType())
)

df_with_conversion = df_with_conversion.withColumn(
    "taxa_conversao_entre_etapas",
    F.round((col("taxa_conversao_autosservico_nivel1") + col("taxa_conversao_nivel1_nivel2")) / 2, 2).cast(FloatType())
)

print("Taxas de conversão calculadas")

### Cálculo de Custo por Etapa

Cálculo de custo por etapa utilizando valores reais da tabela custos.

In [0]:
df_with_cost = df_with_conversion.withColumn(
    "custo_por_etapa",
    when(
        col("total_chamados") > 0,
        F.round(col("custo_total") / col("total_chamados"), 2)
    ).otherwise(lit(None)).cast(DecimalType(10, 2))
)

print("Custos calculados")

### Preparação Final

Adição de metadados de processamento e seleção de colunas finais.
Aplicação de schema com constraints.

In [0]:
df_funnel_final = df_with_cost.withColumn("processed_timestamp", current_timestamp())

final_cols_funnel = [
    "data",
    "canal",
    "n_prevencao",
    "n_autosservico",
    "n_nivel1",
    "n_nivel2",
    "taxa_conversao_entre_etapas",
    "custo_por_etapa",
    "processed_timestamp"
]

df_funnel_final = df_funnel_final.select(*[c for c in final_cols_funnel if c in df_funnel_final.columns])

df_funnel_typed = df_funnel_final \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("n_prevencao", col("n_prevencao").cast(IntegerType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_nivel2", col("n_nivel2").cast(IntegerType())) \
    .withColumn("taxa_conversao_entre_etapas", col("taxa_conversao_entre_etapas").cast(FloatType())) \
    .withColumn("custo_por_etapa", col("custo_por_etapa").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros finais: {df_funnel_typed.count():,}")

### Gravação na Camada Gold

Persistência da tabela transformada em formato Delta na camada Gold.
Modo overwrite com overwriteSchema para garantir schema atualizado.

In [0]:
df_funnel_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_funnel)

gold_table_funnel = spark.table(tgt_table_funnel)
final_count_funnel = gold_table_funnel.count()
duplicates_funnel = gold_table_funnel.groupBy("data", "canal").count().filter(col("count") > 1).count()

print(f"Salvo em: {tgt_table_funnel}")
print(f"Contagem final: {final_count_funnel:,}")
print(f"Duplicatas na Gold: {duplicates_funnel}")

display(gold_table_funnel.limit(10))

### Relatório de Transformação

Resumo estatístico da transformação Silver para Gold.
Métricas de qualidade e distribuição por canal.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FUNNEL_METRICS")
print("="*80)
print(f"Registros Silver (origem chamados): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_funnel:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)